# Comprehensive Sales Analysis - Customer Data Insights

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, ttest_ind, chi2_contingency
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


In [3]:
# Set styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("=== COMPREHENSIVE SALES ANALYSIS FRAMEWORK ===")
print("Libraries loaded successfully!\n")


=== COMPREHENSIVE SALES ANALYSIS FRAMEWORK ===
Libraries loaded successfully!



# =============================================================================
# PHASE 1: DATA FOUNDATION & QUALITY ASSESSMENT
# =============================================================================


In [8]:
print("PHASE 1: DATA FOUNDATION & QUALITY ASSESSMENT")
print("=" * 50)

# Load the data
try:
    sales_data = pd.read_csv('sales_data.csv')
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Error: sales_data.csv not found. Please ensure the file is in the correct directory.")
    exit(1)
df = pd.DataFrame(sales_data)


PHASE 1: DATA FOUNDATION & QUALITY ASSESSMENT
Data loaded successfully!


# 1.1 Data Audit

In [15]:
print("1.1 DATA AUDIT")
print("-" * 20)
display(f"Dataset shape: {df.shape}")
print(f"Missing values:\n{df.isnull().sum()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nFirst 5 rows:")
display(df.head())



1.1 DATA AUDIT
--------------------


'Dataset shape: (16, 11)'

Missing values:
Customer_ID           0
Customer_Name         0
Region                0
Total_Spend           0
Purchase_Frequency    0
Marketing_Spend       0
Seasonality_Index     0
Churned               0
Marketing_ROI         0
Spend_per_Purchase    0
Churned_Binary        0
dtype: int64

Data types:
Customer_ID             int64
Customer_Name          object
Region                 object
Total_Spend             int64
Purchase_Frequency      int64
Marketing_Spend         int64
Seasonality_Index     float64
Churned                object
Marketing_ROI         float64
Spend_per_Purchase    float64
Churned_Binary          int64
dtype: object

First 5 rows:


,Customer_ID,Customer_Name,Region,Total_Spend,Purchase_Frequency,Marketing_Spend,Seasonality_Index,Churned,Marketing_ROI,Spend_per_Purchase,Churned_Binary
0,101,John Doe,North,5000,12,2000,1.2,No,2.5,416.666667,0
1,102,Jane Smith,South,3000,8,1500,1.0,Yes,2.0,375.000000,1
2,103,Sam Brown,East,4500,10,1800,1.1,No,2.5,450.000000,0
3,104,Linda Johnson,West,2500,5,1000,0.9,Yes,2.5,500.000000,1
4,105,Michael Lee,North,7000,15,2500,1.3,No,2.8,466.666667,0


In [10]:
# Create derived metrics
df['Marketing_ROI'] = df['Total_Spend'] / df['Marketing_Spend']
df['Spend_per_Purchase'] = df['Total_Spend'] / df['Purchase_Frequency']
df['Churned_Binary'] = df['Churned'].map({'Yes': 1, 'No': 0})

print(f"\nDerived metrics created: Marketing_ROI, Spend_per_Purchase, Churned_Binary")



Derived metrics created: Marketing_ROI, Spend_per_Purchase, Churned_Binary


# 1.2 Exploratory Data Analysis

In [12]:
print("\n1.2 EXPLORATORY DATA ANALYSIS")
print("-" * 30)

# Descriptive statistics
print("Descriptive Statistics:")
display(df.describe())

# Correlation analysis
numerical_cols = ['Total_Spend', 'Purchase_Frequency', 'Marketing_Spend', 'Seasonality_Index', 'Marketing_ROI', 'Spend_per_Purchase']
correlation_matrix = df[numerical_cols].corr()

plt.figure(figsize=(15, 10))



1.2 EXPLORATORY DATA ANALYSIS
------------------------------
Descriptive Statistics:


,Customer_ID,Total_Spend,Purchase_Frequency,Marketing_Spend,Seasonality_Index,Marketing_ROI,Spend_per_Purchase,Churned_Binary
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,108.500000,4137.500000,9.500000,1675.000000,1.043750,2.449538,440.259168,0.500000
std,4.760952,1396.125591,3.224903,484.424057,0.154785,0.197741,45.277209,0.516398
min,101.000000,2500.000000,5.000000,1000.000000,0.800000,2.000000,366.666667,0.000000
25%,104.750000,2975.000000,6.750000,1300.000000,0.900000,2.302335,409.375000,0.000000
50%,108.500000,3900.000000,9.500000,1650.000000,1.050000,2.474937,450.000000,0.500000
75%,112.250000,5075.000000,12.000000,2025.000000,1.200000,2.529762,462.820513,1.000000
max,116.000000,7000.000000,15.000000,2500.000000,1.300000,2.800000,520.000000,1.000000


<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

### 1.3 Handling Missing Values

In [18]:
# Finding missing values
print("\n--- Missing Values Before Handling ---")
print(df.isnull().sum())

if df.isnull().values.any():
    # Fill missing values with mean for numerical columns
    df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())
    # Fill missing values with mode for categorical columns
    df['Churned'] = df['Churned'].fillna(df['Churned'].mode()[0])
    print("\n--- Missing Values After Handling ---")
    print(df.isnull().sum())



--- Missing Values Before Handling ---
Customer_ID           0
Customer_Name         0
Region                0
Total_Spend           0
Purchase_Frequency    0
Marketing_Spend       0
Seasonality_Index     0
Churned               0
Marketing_ROI         0
Spend_per_Purchase    0
Churned_Binary        0
dtype: int64


### 1.4 Handling Duplicates

In [20]:
# Handling Duplicate Values
print("\n--- Duplicate Rows Before Handling ---")
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Identify and display duplicate rows based on Customer_ID (optional, for inspection)
# Assuming Customer_ID should be unique. If entire row duplicates, df.duplicated() without subset is fine.
if df.duplicated(subset='Customer_ID').any():
    print("Duplicate Customer_ID entries identified:")
    print(df[df.duplicated(subset='Customer_ID', keep=False)].sort_values(by='Customer_ID'))

# Remove duplicate rows based on 'Customer_ID', keeping the first occurrence
df.drop_duplicates(subset='Customer_ID', keep='first', inplace=True)
print("\nDuplicate rows (based on 'Customer_ID') removed.")
print(f"Number of duplicate rows after removal: {df.duplicated().sum()}")

print(f"New dataset shape after duplicate removal: Rows = {df.shape[0]}, Columns = {df.shape[1]}")


--- Duplicate Rows Before Handling ---
Number of duplicate rows: 0

Duplicate rows (based on 'Customer_ID') removed.
Number of duplicate rows after removal: 0
New dataset shape after duplicate removal: Rows = 16, Columns = 11


### 1.5 Datatype Conversion

In [21]:
# Markdown heading for this section
# ### 5. Data Type Conversion

print("\n--- Data Types Before Conversion ---")
print(df.dtypes)

# Convert 'Customer_ID' to integer (handle non-numeric first by coercing to NaN, though not expected from this dataset)
if 'Customer_ID' in df.columns:
    df['Customer_ID'] = pd.to_numeric(df['Customer_ID'], errors='coerce')
    if df['Customer_ID'].isnull().any():
        print("Warning: Non-numeric Customer_ID values detected and converted to NaN. Review data source.")
        # Decide how to handle: drop rows, or impute with unique new IDs. For now, drop.
        df.dropna(subset=['Customer_ID'], inplace=True)
    df['Customer_ID'] = df['Customer_ID'].astype(int)
    print("'Customer_ID' converted to int.")


# Convert numerical columns to appropriate types and handle NaNs after conversion
numerical_cols_to_convert = ['Total_Spend', 'Purchase_Frequency', 'Marketing_Spend', 'Seasonality_Index']
for col in numerical_cols_to_convert:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # Impute NaNs after conversion with the median
        if df[col].isnull().any():
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"'{col}' column: Non-numeric values converted to NaN and then imputed with median ({median_val}).")
        else:
            print(f"'{col}' column converted to numeric.")

# Convert 'Churned' to numerical (0 for 'No', 1 for 'Yes')
if 'Churned' in df.columns:
    # Ensure all values are handled, case-insensitively
    df['Churned'] = df['Churned'].str.strip().str.lower().map({'yes': 1, 'no': 0})
    # Handle any unmapped values (e.g., set to NaN and then impute or drop)
    if df['Churned'].isnull().any():
        print("Warning: Some 'Churned' values could not be mapped to 0/1. These will be set to the mode.")
        df['Churned'].fillna(df['Churned'].mode()[0], inplace=True)
    df['Churned'] = df['Churned'].astype(int)
    print("'Churned' column converted to 0 (No) / 1 (Yes).")


print("\n--- Data Types After Conversion ---")
print(df.dtypes)


--- Data Types Before Conversion ---
Customer_ID             int64
Customer_Name          object
Region                 object
Total_Spend             int64
Purchase_Frequency      int64
Marketing_Spend         int64
Seasonality_Index     float64
Churned                object
Marketing_ROI         float64
Spend_per_Purchase    float64
Churned_Binary          int64
dtype: object
'Customer_ID' converted to int.
'Total_Spend' column converted to numeric.
'Purchase_Frequency' column converted to numeric.
'Marketing_Spend' column converted to numeric.
'Seasonality_Index' column converted to numeric.
'Churned' column converted to 0 (No) / 1 (Yes).

--- Data Types After Conversion ---
Customer_ID             int32
Customer_Name          object
Region                 object
Total_Spend             int64
Purchase_Frequency      int64
Marketing_Spend         int64
Seasonality_Index     float64
Churned                 int32
Marketing_ROI         float64
Spend_per_Purchase    float64
Churned_Bina

### 1.6 Handling Outliers

In [22]:
print("\n--- Outlier Detection and Handling ---")

# Define numerical columns for outlier handling
numerical_cols_for_outliers = ['Total_Spend', 'Purchase_Frequency', 'Marketing_Spend', 'Seasonality_Index']

for col in numerical_cols_for_outliers:
    if col in df.columns:
        # For 'Purchase_Frequency', replace negative values with median
        if col == 'Purchase_Frequency':
            median_val = df[col].median()
            initial_negative = df[df[col] < 0].shape[0]
            if initial_negative > 0:
                df[col] = df[col].apply(lambda x: median_val if x < 0 else x)
                print(f"Negative values in '{col}' ({initial_negative} instances) handled by imputing with median.")

        # Using IQR method for outlier detection and capping
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
        initial_outliers = df[outliers_mask].shape[0]

        if initial_outliers > 0:
            # Cap outliers
            df[col] = df[col].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
            print(f"Outliers in '{col}' ({initial_outliers} instances) detected using IQR and handled (capped).")
            print(f"  Lower Bound: {lower_bound:.2f}, Upper Bound: {upper_bound:.2f}")
        else:
            print(f"No significant outliers detected in '{col}' using IQR method.")

print("\n--- Descriptive Statistics After Outlier Handling (Numerical Columns) ---")
print(df[numerical_cols_for_outliers].describe())


--- Outlier Detection and Handling ---
No significant outliers detected in 'Total_Spend' using IQR method.
No significant outliers detected in 'Purchase_Frequency' using IQR method.
No significant outliers detected in 'Marketing_Spend' using IQR method.
No significant outliers detected in 'Seasonality_Index' using IQR method.

--- Descriptive Statistics After Outlier Handling (Numerical Columns) ---
       Total_Spend  Purchase_Frequency  Marketing_Spend  Seasonality_Index
count    16.000000           16.000000        16.000000          16.000000
mean   4137.500000            9.500000      1675.000000           1.043750
std    1396.125591            3.224903       484.424057           0.154785
min    2500.000000            5.000000      1000.000000           0.800000
25%    2975.000000            6.750000      1300.000000           0.900000
50%    3900.000000            9.500000      1650.000000           1.050000
75%    5075.000000           12.000000      2025.000000           1.200

### 1.7 Data Normalisation/Standardisation

In [23]:
# Data Standardization/Normalization (Feature Scaling)

print("\n--- Feature Scaling (Standardization) ---")

from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Define numerical columns to scale (excluding IDs and binary target 'Churned')
numerical_cols_to_scale = ['Total_Spend', 'Purchase_Frequency', 'Marketing_Spend', 'Seasonality_Index']

# Check if columns exist before scaling
existing_numerical_cols_to_scale = [col for col in numerical_cols_to_scale if col in df.columns]

if existing_numerical_cols_to_scale:
    # Apply standardization
    df[existing_numerical_cols_to_scale] = scaler.fit_transform(df[existing_numerical_cols_to_scale])
    print(f"Columns {existing_numerical_cols_to_scale} have been standardized.")
    print("\n--- First 5 rows after standardization (scaled columns) ---")
    print(df[existing_numerical_cols_to_scale].head())
else:
    print("No numerical columns found for standardization.")


--- Feature Scaling (Standardization) ---
Columns ['Total_Spend', 'Purchase_Frequency', 'Marketing_Spend', 'Seasonality_Index'] have been standardized.

--- First 5 rows after standardization (scaled columns) ---
   Total_Spend  Purchase_Frequency  Marketing_Spend  Seasonality_Index
0     0.638042            0.800641         0.692902           1.042572
1    -0.841475           -0.480384        -0.373101          -0.291920
2     0.268162            0.160128         0.266501           0.375326
3    -1.211354           -1.441153        -1.439105          -0.959166
4     2.117558            1.761410         1.758906           1.709818


### 1.8 Handling Categorical Data

In [24]:
# 8. Handling Categorical Data

print("\n--- Categorical Data Encoding ---")

# Apply One-Hot Encoding to 'Region'
# 'Churned' has already been converted to 0/1 in the data type conversion step.
categorical_cols_to_encode = ['Region']

# Check if columns exist before encoding
existing_categorical_cols_to_encode = [col for col in categorical_cols_to_encode if col in df.columns]

if existing_categorical_cols_to_encode:
    # Using get_dummies for one-hot encoding, drop_first avoids multicollinearity
    df = pd.get_dummies(df, columns=existing_categorical_cols_to_encode, drop_first=True)
    print(f"Categorical columns {existing_categorical_cols_to_encode} have been One-Hot Encoded.")
    print("\n--- First 5 rows after One-Hot Encoding ---")
    print(df.head())
else:
    print("No specified categorical columns found for encoding.")


--- Categorical Data Encoding ---
Categorical columns ['Region'] have been One-Hot Encoded.

--- First 5 rows after One-Hot Encoding ---
   Customer_ID  Customer_Name  Total_Spend  Purchase_Frequency  \
0          101       John Doe     0.638042            0.800641   
1          102     Jane Smith    -0.841475           -0.480384   
2          103      Sam Brown     0.268162            0.160128   
3          104  Linda Johnson    -1.211354           -1.441153   
4          105    Michael Lee     2.117558            1.761410   

   Marketing_Spend  Seasonality_Index  Churned  Marketing_ROI  \
0         0.692902           1.042572        0            2.5   
1        -0.373101          -0.291920        1            2.0   
2         0.266501           0.375326        0            2.5   
3        -1.439105          -0.959166        1            2.5   
4         1.758906           1.709818        0            2.8   

   Spend_per_Purchase  Churned_Binary  Region_North  Region_South  Region_